# Exploring sequence of Tn10 

In [1]:
import wgregseq

First we read the FASTA file obtained from Genebank.

In [ ]:
with open ("tn10.fasta", "r") as file:
    data = file.read().split('\n')[1:]
    sequence = "".join(data)

Organization of tetR/tetA regulation:
- two operators that can be bound independently by TetR
- tetA is repressed by both tetO1 and tetO2
- tetR is repressed only by tetO1
- Affinity of tetO2 to TetR is about twice as high as tetO1

From Genebank, we can find the positions for *tetA* and *tetR*. The repressor gene is reversed, so we will have to obtain the complementary sequence in case we are interested in the actual sequences.

In [ ]:
# Exact positions from Genebank
tetR_pos = [4702, 5328]
tetA_pos = [5407, 6612]

tetA = sequence[tetA_pos[0]-1:tetA_pos[1]]
tetR = wgregseq.complement_seq(sequence[tetA_pos[0]-1:tetA_pos[1]])

In [ ]:
# Positions taken from Bertram 2008
tetO2_pos = [tetA_pos[0] - 28, tetA_pos[0] - 10]
tetO1_pos = [tetA_pos[0] - 58, tetA_pos[0] - 40]

tetO2 = sequence[tetO2_pos[0]:tetO2_pos[1]+1]
tetO1 = sequence[tetO1_pos[0]:tetO1_pos[1]+1]

In [ ]:
# TSS estimated from -10 region
tetA_TSS = tetA_pos[0] - 29
tetR_TSS1 = tetO2_pos[0] - 19
tetR_TSS2 = tetO1_pos[0] - 8

In [ ]:
sequence[tetA_TSS:tetA_TSS+3]

In [ ]:
sequence[tetR_TSS2-4:tetR_TSS2+1]

In [ ]:
sequence[tetR_TSS1:tetR_TSS1+4]

In [ ]:
try:
    
except ValueError:
    2

In [6]:
EcoRI.site

'GAATTC'

In [9]:
[seq for seq in CommOnly]

[BfuAI,
 BstBAI,
 BseBI,
 CviJI,
 AgeI,
 Alw44I,
 AccI,
 Fsp4HI,
 KflI,
 BstKTI,
 PspN4I,
 BspT104I,
 PscI,
 TauI,
 BbsI,
 EciI,
 FatI,
 MauBI,
 Eco24I,
 BbvI,
 BspFNI,
 BmrFI,
 AcoI,
 EcoNI,
 FblI,
 BspMAI,
 PmaCI,
 Bsp1407I,
 Zsp2I,
 AfiI,
 BsuI,
 DpnII,
 Ksp22I,
 DrdI,
 TspGWI,
 BshFI,
 VspI,
 Bme18I,
 PluTI,
 GsaI,
 BstNI,
 Eco130I,
 MwoI,
 SduI,
 AloI,
 SwaI,
 OliI,
 Bsp19I,
 XmiI,
 Sse8387I,
 BseMI,
 Cfr10I,
 Hin6I,
 VneI,
 BstMCI,
 PciSI,
 MmeI,
 BsaBI,
 SgrAI,
 AxyI,
 KpnI,
 BlpI,
 Eco47I,
 DraIII,
 SfoI,
 Msp20I,
 BshTI,
 SmoI,
 Sfr303I,
 BssHII,
 FauNDI,
 TatI,
 Bsh1285I,
 LguI,
 BbrPI,
 NotI,
 SapI,
 AflII,
 AvaI,
 BtgI,
 AgsI,
 BslI,
 Bse1I,
 PsiI,
 Mox20I,
 Bse118I,
 TfiI,
 Rsr2I,
 RigI,
 AleI,
 BseCI,
 Csp6I,
 SpeI,
 BlsI,
 NcoI,
 Eam1104I,
 Pfl23II,
 FokI,
 Eco47III,
 AflIII,
 BstPI,
 BciVI,
 NlaIV,
 KspAI,
 BfrI,
 SinI,
 Psp6I,
 Ple19I,
 BspCNI,
 BaeI,
 BclI,
 BstZI,
 SfuI,
 BstV1I,
 MluI,
 PagI,
 BtgZI,
 DinI,
 BstDEI,
 AsuNHI,
 HpaI,
 FspAI,
 RsrII,
 PfeI,
 KspI,
 Alw

In [ ]:
any([False])

In [ ]:
from Bio.Seq import Seq
my_seq = Seq('AAAAAAAAAAAAAA')

In [ ]:
type(my_seq)